In [ ]:
def key_gen(K):
  key_compression = [14, 17, 11, 24, 1,  5,  3,  28, 15, 6,  21, 10, 23, 19, 12, 4, 26, 8,  16, 7,  27, 20, 13, 2,
                     41, 52, 31, 37, 47, 55, 30, 40, 51, 45, 33, 48, 44, 49, 39, 56, 34, 53, 46, 42, 50, 36, 29, 32 ]
  key_parity = [57, 49, 41, 33, 25, 17, 9,  1,  58, 50, 42, 34,
            26, 18, 10, 2,  59, 51, 43, 35, 27, 19, 11, 3,
            60, 52, 44, 36, 63, 55, 47, 39, 31, 23, 15, 7,
            62, 54, 46, 38, 30, 22, 14, 6,  61, 53, 45, 37,
            29, 21, 13, 5,  28, 20, 12, 4 ]
  keys = []
  parity_drop=""
  for i in range(56):
    parity_drop+=K[key_parity[i]-1]
  for i in range(1,17):
    left = parity_drop[0:28]
    right = parity_drop[28:]
    if(i in [1,2,9,16]):
      left = left[1:]+left[0]
      right = right[1:]+right[0]
    else:
      left = left[2:]+left[0:2]
      right = right[2:]+right[0:2]
    key = left+right
    parity_drop=key
    compressed_key=""
    for j in range(48):
      compressed_key+= key[key_compression[j]-1]
    keys.append(compressed_key)
  return keys

In [ ]:
def XOR(s1, s2):
  if(len(s1)!=len(s2)):
    raise Exception("length not matching")
  ans =""
  for i in range(len(s1)):
    if(s1[i]==s2[i]):
      ans+='0'
    else:
      ans+='1'
  return ans

def F_box(K,C):
  expansion_box = [32, 1,  2,  3,  4,  5,  4,  5,  6,  7,  8,  9, 8,  9,  10, 11, 12, 13, 12, 13, 14, 15, 16, 17,
            16, 17, 18, 19, 20, 21, 20, 21, 22, 23, 24, 25, 24, 25, 26, 27, 28, 29, 28, 29, 30, 31, 32, 1]
  S_boxes =[
        [14, 4,  13, 1, 2,  15, 11, 8,  3,  10, 6,  12, 5, 9,  0,  7,  
         0, 15, 7,  4,  14, 2,  13, 1,  10, 6, 12, 11, 9,  5, 3,  8,  
         4,  1,  14, 8,  13, 6,  2, 11, 15, 12, 9, 7,  3,  10, 5,  0,  
         15, 12, 8,  2, 4,  9,  1,  7, 5,  11, 3,  14, 10, 0,  6,  13 ],

        [15, 1,  8,  14, 6,  11, 3, 4,  9,  7,  2,  13, 12, 0,  5,  10, 
         3,  13, 4,  7, 15, 2,  8,  14, 12, 0, 1,  10, 6,  9,  11, 5,  
         0, 14, 7,  11, 10, 4,  13, 1,  5,  8,  12, 6,  9,  3, 2,  15, 
         13, 8,  10, 1, 3,  15, 4,  2,  11, 6,  7, 12, 0,  5,  14, 9 ],

         [10, 0,  9,  14, 6,  3,  15, 5,  1,  13, 12, 7,  11, 4,  2,  8,  
          13, 7,  0,  9,  3,  4, 6,  10, 2,  8,  5,  14, 12, 11, 15, 1,  
          13, 6,  4,  9,  8,  15, 3,  0,  11, 1,  2,  12, 5,  10, 14, 7,  
          1,  10, 13, 0,  6,  9,  8, 7,  4,  15, 14, 3,  11, 5,  2,  12 ],

        [ 7,  13, 14, 3,  0,  6,  9,  10, 1,  2, 8,  5,  11, 12, 4,  15, 
         13, 8,  11, 5,  6,  15, 0, 3,  4,  7, 2,  12, 1,  10, 14, 9,  
         10, 6,  9,  0, 12, 11, 7, 13, 15, 1,  3,  14, 5,  2,  8,  4,  
         3, 15, 0,  6, 10, 1,  13, 8,  9,  4,  5,  11, 12, 7, 2,  14 ],

        [ 2,  12, 4, 1,  7,  10, 11, 6, 8,  5,  3,  15, 13, 0,  14, 9, 
         14, 11, 2,  12, 4, 7,  13, 1,  5,  0, 15, 10, 3, 9,  8,  6,  
         4,  2, 1,  11, 10, 13, 7, 8,  15, 9, 12, 5,  6,  3,  0, 14, 
         11, 8,  12, 7, 1,  14, 2, 13, 6,  15, 0,  9, 10, 4,  5,  3 ],

        [ 12, 1,  10, 15, 9,  2,  6,  8,  0,  13, 3, 4, 14, 7,  5,  11, 
         10, 15, 4,  2,  7,  12, 9,  5, 6, 1, 13, 14, 0,  11, 3,  8,  
         9,  14, 15, 5,  2, 8, 12, 3,  7,  0,  4,  10, 1,  13, 11, 6,  
         4,  3, 2, 12, 9,  5,  15, 10, 11, 14, 1,  7,  6,  0,  8, 13 ],

        [ 4,  11, 2,  14, 15, 0,  8, 13, 3,  12, 9,  7,  5, 10, 6,  1,  
         13, 0,  11, 7, 4,  9,  1,  10, 14, 3, 5,  12, 2,  15, 8,  6,  
         1, 4,  11, 13, 12, 3,  7, 14, 10, 15, 6,  8,  0,  5, 9,  2,  
         6,  11, 13, 8, 1,  4,  10, 7,  9,  5,  0, 15, 14, 2,  3,  12 ],

        [13, 2,  8, 4,  6,  15, 11, 1,  10, 9, 3, 14, 5, 0,  12, 7, 
         1,  15, 13, 8,  10, 3,  7, 4, 12, 5, 6,  11, 0, 14, 9,  2,  
         7,  11, 4,  1, 9, 12, 14,2,  0,  6, 10, 13, 15, 3,  5,  8,  
         2, 1, 14, 7, 4,  10, 8, 13, 15, 12, 9,  0,  3,  5, 6, 11 ]
    ]
  straight_permutation = [ 16, 7, 20, 21, 29, 12, 28, 17, 1,  15, 23,
            26, 5, 18, 31, 10, 2,  8,  24, 14, 32, 27,
            3,  9, 19, 13, 30, 6,  22, 11, 4,  25]

  C_expand=""
  for i in range(48):
    C_expand+=C[expansion_box[i]-1]
  xored_C = XOR(C_expand,K)
  after_s_box=""
  for block in range(8):
    bits = xored_C[6*block:6*block+6]
    row = int(bits[0]+bits[5],2)
    col = int(bits[1:5],2)
    new_bits = '{0:b}'.format(S_boxes[block][16*row+col])
    n = len(new_bits)
    for i in range(4-n):
      new_bits = '0'+new_bits
    after_s_box+=new_bits
  ans =""
  for i in range(32):
    ans+=after_s_box[straight_permutation[i]-1]
  return ans

def initial_permutation(P):
  initial_lookup = [58, 50, 42, 34, 26, 18, 10, 2,  60, 52, 44, 36, 28, 20, 12, 4,  62, 54, 46, 38, 30, 22,
            14, 6,  64, 56, 48, 40, 32, 24, 16, 8,  57, 49, 41, 33, 25, 17, 9,  1,  59, 51, 43, 35,
            27, 19, 11, 3,  61, 53, 45, 37, 29, 21, 13, 5,  63, 55, 47, 39, 31, 23, 15, 7 ]
  ans=""
  for i in range(64):
    ans+=P[initial_lookup[i]-1]
  return ans

def DES_round(K,C):
  LE_i = C[0:32]
  RE_i = C[32:64]
  F_out = F_box(K,RE_i)
  LE_i_1 = RE_i
  RE_i_1 = XOR(F_out,LE_i)
  return (LE_i_1+RE_i_1)

def swap_32_bit(C):
  return C[32:64]+C[0:32]

def inverse_initial_permutation(C):
  final_lookup = [40, 8,  48, 16, 56, 24, 64, 32, 39, 7,  47,15, 55, 23, 63, 31, 38, 6,  46, 14, 54, 22,
            62, 30, 37, 5,  45, 13, 53, 21, 61, 29, 36, 4,  44, 12, 52, 20, 60, 28, 35, 3,  43, 11,
            51, 19, 59, 27, 34, 2,  42, 10, 50, 18, 58, 26, 33, 1,  41, 9,  49, 17, 57, 25 ]
  ans=""
  for i in range(64):
    ans+=C[final_lookup[i]-1]
  return ans

## Encryption

In [ ]:
def DES_encryption(K, P):
  intermediate_keys=key_gen(K)
  intermediate_ciphers=[]

  intermediate_ciphers.append(initial_permutation(P))

  for round in range(16):
    intermediate_ciphers.append(DES_round(intermediate_keys[round],intermediate_ciphers[-1]))

  intermediate_ciphers.append(swap_32_bit(intermediate_ciphers[-1]))

  intermediate_ciphers.append(inverse_initial_permutation(intermediate_ciphers[-1]))
  
  return intermediate_ciphers

## Decryption

In [ ]:
def DES_decryption(K, C):
  intermediate_keys=key_gen(K)
  
  intermediate_keys.reverse()

  intermediate_ciphers=[]

  intermediate_ciphers.append(initial_permutation(C))

  

  for round in range(16):
    intermediate_ciphers.append(DES_round(intermediate_keys[round],intermediate_ciphers[-1]))

  intermediate_ciphers.append(swap_32_bit(intermediate_ciphers[-1]))
  intermediate_ciphers.append(inverse_initial_permutation(intermediate_ciphers[-1]))
  
  return intermediate_ciphers

Example1

In [ ]:
K='0000111100010101011100011100100101000111110110011110100001011001' #key
P='0000001001000110100010101100111011101100101010000110010000100000' #plaintext
print("Plaintext: ", hex(int(P[:32],2)), hex(int(P[32:],2)))

C=DES_encryption(K,P)

print("After encryption")
print("C after initial_permutation: ", hex(int(C[0][:32],2)), hex(int(C[0][32:],2)))
for i in range(1,17):
  print("C after round ",i,": ", hex(int(C[i][:32],2)), hex(int(C[i][32:],2)))
print("C after 32 bit swap: ", hex(int(C[17][:32],2)), hex(int(C[17][32:],2)))
print("C after inverse_initial_permutation: ", hex(int(C[18][:32],2)), hex(int(C[18][32:],2)))
print("final C: ", hex(int(C[-1][:32],2)), hex(int(C[-1][32:],2)))


print("----------")


print("After decryption")
P2=DES_decryption(K, C[-1])
print("P2 after initial_permutation: ", hex(int(P2[0][:32],2)), hex(int(P2[0][32:],2)))
for i in range(1,17):
  print("P2 after round ",i,": ", hex(int(P2[i][:32],2)),  hex(int(P2[i][32:],2) ))
print("P2 after 32 bit swap: ", hex(int(P2[17][:32],2)), hex(int(P2[17][32:],2)))
print("P2 after inverse_initial_permutation: ", hex(int(P2[18][:32],2)), hex(int(P2[18][32:],2)))

Plaintext:  0x2468ace 0xeca86420
After encryption
C after initial_permutation:  0x5a005a00 0x3cf03c0f
C after round  1 :  0x3cf03c0f 0xbad22845
C after round  2 :  0xbad22845 0x99e9b723
C after round  3 :  0x99e9b723 0xbae3b9e
C after round  4 :  0xbae3b9e 0x42415649
C after round  5 :  0x42415649 0x18b3fa41
C after round  6 :  0x18b3fa41 0x9616fe23
C after round  7 :  0x9616fe23 0x67117cf2
C after round  8 :  0x67117cf2 0xc11bfc09
C after round  9 :  0xc11bfc09 0x887fbc6c
C after round  10 :  0x887fbc6c 0x600f7e8b
C after round  11 :  0x600f7e8b 0xf596506e
C after round  12 :  0xf596506e 0x738538b8
C after round  13 :  0x738538b8 0xc6a62c4e
C after round  14 :  0xc6a62c4e 0x56b0bd75
C after round  15 :  0x56b0bd75 0x75e8fd8f
C after round  16 :  0x75e8fd8f 0x25896490
C after 32 bit swap:  0x25896490 0x75e8fd8f
C after inverse_initial_permutation:  0xda02ce3a 0x89ecac3b
final C:  0xda02ce3a 0x89ecac3b
----------
After decryption
P2 after initial_permutation:  0x25896490 0x75e8fd8f
P2 a

In [ ]:
KK =key_gen(K)
round =0
for k in KK:
  round+=1
  st=""
  for i in range(0,48,6):
    f=str(hex(int(k[i:i+6],2)))[2:]
    if(len(f)==1):
      f="0"+f
    st+=f
  print("round key ",round,": ",st)

round key  1 :  1e030f03080d2930
round key  2 :  0a31293432242318
round key  3 :  23072318201d0c1d
round key  4 :  05261d3824311a20
round key  5 :  3325340136002c25
round key  6 :  123a2d0d04262a1c
round key  7 :  021f120b1c130611
round key  8 :  1c10372a2832002b
round key  9 :  04292a380c341f03
round key  10 :  2703212607280403
round key  11 :  2826390c31261504
round key  12 :  12071c241a0a0f08
round key  13 :  300935393c0d100b
round key  14 :  311e09231321182a
round key  15 :  283d3e0227072528
round key  16 :  2921080b13143025


Example2

In [ ]:
K='0111001000110110100010101101011110110001111000011011000010010001' #key
P='1000000101110010101010111110000100011011000111000001110000010010' #plaintext
print("Plaintext: ", hex(int(P[:32],2)), hex(int(P[32:],2)))

C=DES_encryption(K,P)

print("After encryption")
print("C after initial_permutation: ", hex(int(C[0][:32],2)), hex(int(C[0][32:],2)))
for i in range(1,17):
  print("C after round ",i,": ", hex(int(C[i][:32],2)), hex(int(C[i][32:],2)))
print("C after 32 bit swap: ", hex(int(C[17][:32],2)), hex(int(C[17][32:],2)))
print("C after inverse_initial_permutation: ", hex(int(C[18][:32],2)), hex(int(C[18][32:],2)))
print("final C: ", hex(int(C[-1][:32],2)), hex(int(C[-1][32:],2)))


print("----------")


print("After decryption")
P2=DES_decryption(K, C[-1])
print("P2 after initial_permutation: ", hex(int(P2[0][:32],2)), hex(int(P2[0][32:],2)))
for i in range(1,17):
  print("P2 after round ",i,": ", hex(int(P2[i][:32],2)),  hex(int(P2[i][32:],2) ))
print("P2 after 32 bit swap: ", hex(int(P2[17][:32],2)), hex(int(P2[17][32:],2)))
print("P2 after inverse_initial_permutation: ", hex(int(P2[18][:32],2)), hex(int(P2[18][32:],2)))

Plaintext:  0x8172abe1 0x1b1c1c12
After encryption
C after initial_permutation:  0xaf2601d 0xd0e7496
C after round  1 :  0xd0e7496 0xe2de1b9
C after round  2 :  0xe2de1b9 0xb78e1533
C after round  3 :  0xb78e1533 0xe36d86ab
C after round  4 :  0xe36d86ab 0x4e77ba7f
C after round  5 :  0x4e77ba7f 0x768297f1
C after round  6 :  0x768297f1 0xdb05c8be
C after round  7 :  0xdb05c8be 0xf7064566
C after round  8 :  0xf7064566 0x4a58c712
C after round  9 :  0x4a58c712 0xf4ed7f1
C after round  10 :  0xf4ed7f1 0x7cc5e2b2
C after round  11 :  0x7cc5e2b2 0x57b918c1
C after round  12 :  0x57b918c1 0xc9455767
C after round  13 :  0xc9455767 0xb1d7ae46
C after round  14 :  0xb1d7ae46 0xda742580
C after round  15 :  0xda742580 0x6fe5d0de
C after round  16 :  0x6fe5d0de 0x3f0cd194
C after 32 bit swap:  0x3f0cd194 0x6fe5d0de
C after inverse_initial_permutation:  0xe4c2f3d2 0x4fe0ae2f
final C:  0xe4c2f3d2 0x4fe0ae2f
----------
After decryption
P2 after initial_permutation:  0x3f0cd194 0x6fe5d0de
P2 after

In [ ]:
KK =key_gen(K)
round =0
for k in KK:
  round+=1
  st=""
  for i in range(0,48,6):
    f=str(hex(int(k[i:i+6],2)))[2:]
    if(len(f)==1):
      f="0"+f
    st+=f
  print("round key ",round,": ",st)

round key  1 :  17393c0911121211
round key  2 :  3e322629280b0b01
round key  3 :  261a1a3d1c281823
round key  4 :  35113a1e0720340a
round key  5 :  1d2b2b2403070510
round key  6 :  262f3126181e0120
round key  7 :  3a061d3e3808300a
round key  8 :  390d1c2d2301181e
round key  9 :  3f29172222111420
round key  10 :  3e2c093b12062824
round key  11 :  2b1d0c1e1c04221c
round key  12 :  09212f1f2011021b
round key  13 :  0f37213121330821
round key  14 :  233637380c202d24
round key  15 :  37261e1d04082616
round key  16 :  0f3436272102070c
